# Linear symbolic MAOOAM example 

Testing platform for the symbolic equation version of the qgs model

In [ ]:
import sys, os
sys.path.extend([os.path.abspath('../')])

In [ ]:
import numpy as np
import sympy as sy
import sparse as sp
import math

In [ ]:
from qgs.params.params import QgParams
from qgs.functions.tendencies import create_tendencies

In [ ]:
model_parameters = QgParams(dynamic_T=False)

In [ ]:
model_parameters.set_atmospheric_channel_fourier_modes(2, 2, mode="symbolic")
# Mode truncation at the wavenumber 2 in the x and at the 
# wavenumber 4 in the y spatial coordinates for the ocean
model_parameters.set_oceanic_basin_fourier_modes(2, 4, mode="symbolic")

In [ ]:
model_parameters.symbolic_params

In [ ]:
from qgs.inner_products.symbolic import AtmosphericSymbolicInnerProducts, OceanicSymbolicInnerProducts
from qgs.tensors.qgtensor import QgsTensor, QgsTensorDynamicT, QgsTensorT4
from qgs.tensors.symbolic_qgtensor import SymbolicTensorLinear, SymbolicTensorDynamicT
from qgs.tensors.symbolic_qgtensor import _symbolic_tensordot, _shift_dict_keys, _add_to_dict

In [ ]:
atm_loaded = AtmosphericSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True) 
ocn_loaded = OceanicSymbolicInnerProducts(model_parameters, stored=False, return_symbolic=True)

In [ ]:
atm_loaded.load_from_file('temp_atm_sym_lin.ip')
ocn_loaded.load_from_file('temp_ocn_sym_lin.ip')

In [ ]:
model_parameters.print_params()

## Outputting model equations

In [ ]:
from qgs.functions.symbolic_output import create_symbolic_equations, translate_equations, equation_as_function, format_equations

Calculating the functions and tensor

In [ ]:
%%time
funcs, = create_symbolic_equations(model_parameters, continuation_variables={'eps'})

In [ ]:
print(funcs)

Below function has the **kwargs input removed as I cannot get this to work with numba

In [ ]:
from numba import njit

In [ ]:
@njit
def f(t, U):
	#Tendency function of the qgs model
	U_out = np.empty_like(U)
	n = 1.3
	epsilon = 0.76
	U_out[0] = -0.05*U[0] + 0.05*U[10] + 0.00716448960313446*U[21]*(1.33333333333333*n**2 + 21.3333333333333) + 0.00716448960313446*U[23]*(0.533333333333333*n**2 + 34.1333333333333)
	U_out[1] = U[0]*U[2]*(-1.20042175487614*n + 1.20042175487614*n/(n**2 + 1.0)) + U[10]*U[12]*(-1.20042175487614*n + 1.20042175487614*n/(n**2 + 1.0)) + 0.05*U[11] + U[13]*U[15]*(-1.92067480780183*n*(n**2 + 4.0)/(n**2 + 1.0) + 7.68269923120731*n/(n**2 + 1.0)) + U[14]*U[17]*(1.5*n*(n**2 + 4.0)/(n**2 + 1.0) + (-6.0*n**3 - 1.5*n)/(n**2 + 1.0)) + U[15]*U[16]*(-1.5*n*(n**2 + 4.0)/(n**2 + 1.0) + (6.0*n**3 + 1.5*n)/(n**2 + 1.0)) - 0.05*U[1] - 0.0159154943091895*U[20]*(0.333333333333333*n**2 + 1.33333333333333)/(n**2 + 1.0) + 0.249850774084608*U[2]*n/(n**2 + 1.0) + U[3]*U[5]*(-1.92067480780183*n*(n**2 + 4.0)/(n**2 + 1.0) + 7.68269923120731*n/(n**2 + 1.0)) + U[4]*U[7]*(1.5*n*(n**2 + 4.0)/(n**2 + 1.0) + (-6.0*n**3 - 1.5*n)/(n**2 + 1.0)) + U[5]*U[6]*(-1.5*n*(n**2 + 4.0)/(n**2 + 1.0) + (6.0*n**3 + 1.5*n)/(n**2 + 1.0))
	U_out[2] = U[0]*U[1]*(1.20042175487614*n - 1.20042175487614*n/(n**2 + 1.0)) + U[10]*U[11]*(1.20042175487614*n - 1.20042175487614*n/(n**2 + 1.0)) + 0.05*U[12] + U[13]*U[14]*(1.92067480780183*n*(n**2 + 4.0)/(n**2 + 1.0) - 7.68269923120731*n/(n**2 + 1.0)) + U[14]*U[16]*(-1.5*n*(n**2 + 4.0)/(n**2 + 1.0) + (6.0*n**3 + 1.5*n)/(n**2 + 1.0)) + U[15]*U[17]*(-1.5*n*(n**2 + 4.0)/(n**2 + 1.0) + (6.0*n**3 + 1.5*n)/(n**2 + 1.0)) - 0.249850774084608*U[1]*n/(n**2 + 1.0) - 0.05*U[24]*(-1.0*n**2 - 1.0)/(n**2 + 1.0) - 0.05*U[2] + U[3]*U[4]*(1.92067480780183*n*(n**2 + 4.0)/(n**2 + 1.0) - 7.68269923120731*n/(n**2 + 1.0)) + U[4]*U[6]*(-1.5*n*(n**2 + 4.0)/(n**2 + 1.0) + (6.0*n**3 + 1.5*n)/(n**2 + 1.0)) + U[5]*U[7]*(-1.5*n*(n**2 + 4.0)/(n**2 + 1.0) + (6.0*n**3 + 1.5*n)/(n**2 + 1.0))
	U_out[3] = U[11]*U[15]*(-0.480168701950457*n*(n**2 + 1.0) + 0.480168701950457*n*(n**2 + 4.0)) + U[12]*U[14]*(0.480168701950457*n*(n**2 + 1.0) - 0.480168701950457*n*(n**2 + 4.0)) + 0.05*U[13] + U[16]*U[19]*(3.84134961560365*n*(n**2 + 1.0) - 0.960337403900913*n*(4.0*n**2 + 1.0)) + U[17]*U[18]*(-3.84134961560365*n*(n**2 + 1.0) + 0.960337403900913*n*(4.0*n**2 + 1.0)) + U[1]*U[5]*(-0.480168701950457*n*(n**2 + 1.0) + 0.480168701950457*n*(n**2 + 4.0)) - 0.00179112240078361*U[20]*(0.666666666666667*n**2 + 2.66666666666667) + 0.00179112240078361*U[22]*(1.2*n**2 + 43.2) + U[2]*U[4]*(0.480168701950457*n*(n**2 + 1.0) - 0.480168701950457*n*(n**2 + 4.0)) - 0.05*U[3] + U[6]*U[9]*(3.84134961560365*n*(n**2 + 1.0) - 0.960337403900913*n*(4.0*n**2 + 1.0)) + U[7]*U[8]*(-3.84134961560365*n*(n**2 + 1.0) + 0.960337403900913*n*(4.0*n**2 + 1.0))
	U_out[4] = U[0]*U[5]*(-0.960337403900913*n + 0.960337403900913*n/(n**2 + 4.0)) + U[10]*U[15]*(-0.960337403900913*n + 0.960337403900913*n/(n**2 + 4.0)) + U[11]*U[17]*(-1.5*n*(n**2 + 1.0)/(n**2 + 4.0) + (6.0*n**3 + 1.5*n)/(n**2 + 4.0)) + U[12]*U[13]*(-1.92067480780183*n*(n**2 + 1.0)/(n**2 + 4.0) + 7.68269923120731*n/(n**2 + 4.0)) + U[12]*U[16]*(1.5*n*(n**2 + 1.0)/(n**2 + 4.0) + (-6.0*n**3 - 1.5*n)/(n**2 + 4.0)) + 0.05*U[14] + U[1]*U[7]*(-1.5*n*(n**2 + 1.0)/(n**2 + 4.0) + (6.0*n**3 + 1.5*n)/(n**2 + 4.0)) - 0.0159154943091895*U[21]*(0.333333333333333*n**2 + 5.33333333333333)/(n**2 + 4.0) + U[2]*U[3]*(-1.92067480780183*n*(n**2 + 1.0)/(n**2 + 4.0) + 7.68269923120731*n/(n**2 + 4.0)) + U[2]*U[6]*(1.5*n*(n**2 + 1.0)/(n**2 + 4.0) + (-6.0*n**3 - 1.5*n)/(n**2 + 4.0)) - 0.05*U[4] + 0.249850774084608*U[5]*n/(n**2 + 4.0)
	U_out[5] = U[0]*U[4]*(0.960337403900913*n - 0.960337403900913*n/(n**2 + 4.0)) + U[10]*U[14]*(0.960337403900913*n - 0.960337403900913*n/(n**2 + 4.0)) + U[11]*U[13]*(1.92067480780183*n*(n**2 + 1.0)/(n**2 + 4.0) - 7.68269923120731*n/(n**2 + 4.0)) + U[11]*U[16]*(1.5*n*(n**2 + 1.0)/(n**2 + 4.0) + (-6.0*n**3 - 1.5*n)/(n**2 + 4.0)) + U[12]*U[17]*(1.5*n*(n**2 + 1.0)/(n**2 + 4.0) + (-6.0*n**3 - 1.5*n)/(n**2 + 4.0)) + 0.05*U[15] + U[1]*U[3]*(1.92067480780183*n*(n**2 + 1.0)/(n**2 + 4.0) - 7.68269923120731*n/(n**2 + 4.0)) + U[1]*U[6]*(1.5*n*(n**2 + 1.0)/(n**2 + 4.0) + (-6.0*n**3 - 1.5*n)/(n**2 + 4.0)) - 0.05*U[25]*(-1.0*n**2 - 4.0)/(n**2 + 4.0) + U[2]*U[7]*(1.5*n*(n**2 + 1.0)/(n**2 + 4.0) + (-6.0*n**3 - 1.5*n)/(n**2 + 4.0)) - 0.249850774084608*U[4]*n/(n**2 + 4.0) - 0.05*U[5]
	U_out[6] = U[0]*U[7]*(-2.40084350975228*n + 2.40084350975228*n/(4.0*n**2 + 1.0)) + U[10]*U[17]*(-2.40084350975228*n + 2.40084350975228*n/(4.0*n**2 + 1.0)) + U[11]*U[15]*(-1.5*n*(n**2 + 1.0)/(4.0*n**2 + 1.0) + 1.5*n*(n**2 + 4.0)/(4.0*n**2 + 1.0)) + U[12]*U[14]*(-1.5*n*(n**2 + 1.0)/(4.0*n**2 + 1.0) + 1.5*n*(n**2 + 4.0)/(4.0*n**2 + 1.0)) + U[13]*U[19]*(-15.3653984624146*n*(n**2 + 1.0)/(4.0*n**2 + 1.0) + 15.3653984624146*n/(4.0*n**2 + 1.0)) + 0.05*U[16] + U[1]*U[5]*(-1.5*n*(n**2 + 1.0)/(4.0*n**2 + 1.0) + 1.5*n*(n**2 + 4.0)/(4.0*n**2 + 1.0)) - 0.0159154943091895*U[20]*(0.0666666666666667*n**2 + 0.266666666666667)/(4.0*n**2 + 1.0) + U[2]*U[4]*(-1.5*n*(n**2 + 1.0)/(4.0*n**2 + 1.0) + 1.5*n*(n**2 + 4.0)/(4.0*n**2 + 1.0)) + U[3]*U[9]*(-15.3653984624146*n*(n**2 + 1.0)/(4.0*n**2 + 1.0) + 15.3653984624146*n/(4.0*n**2 + 1.0)) - 0.05*U[6] + 0.499701548169216*U[7]*n/(4.0*n**2 + 1.0)
	U_out[7] = U[0]*U[6]*(2.40084350975228*n - 2.40084350975228*n/(4.0*n**2 + 1.0)) + U[10]*U[16]*(2.40084350975228*n - 2.40084350975228*n/(4.0*n**2 + 1.0)) + U[11]*U[14]*(1.5*n*(n**2 + 1.0)/(4.0*n**2 + 1.0) - 1.5*n*(n**2 + 4.0)/(4.0*n**2 + 1.0)) + U[12]*U[15]*(-1.5*n*(n**2 + 1.0)/(4.0*n**2 + 1.0) + 1.5*n*(n**2 + 4.0)/(4.0*n**2 + 1.0)) + U[13]*U[18]*(15.3653984624146*n*(n**2 + 1.0)/(4.0*n**2 + 1.0) - 15.3653984624146*n/(4.0*n**2 + 1.0)) + 0.05*U[17] + U[1]*U[4]*(1.5*n*(n**2 + 1.0)/(4.0*n**2 + 1.0) - 1.5*n*(n**2 + 4.0)/(4.0*n**2 + 1.0)) + U[2]*U[5]*(-1.5*n*(n**2 + 1.0)/(4.0*n**2 + 1.0) + 1.5*n*(n**2 + 4.0)/(4.0*n**2 + 1.0)) + U[3]*U[8]*(15.3653984624146*n*(n**2 + 1.0)/(4.0*n**2 + 1.0) - 15.3653984624146*n/(4.0*n**2 + 1.0)) - 0.499701548169216*U[6]*n/(4.0*n**2 + 1.0) - 0.05*U[7]
	U_out[8] = U[0]*U[9]*(-7.68269923120731*n*(n**2 + 1.0)/(4.0*n**2 + 4.0) + 1.92067480780183*n/(4.0*n**2 + 4.0)) + U[10]*U[19]*(-7.68269923120731*n*(n**2 + 1.0)/(4.0*n**2 + 4.0) + 1.92067480780183*n/(4.0*n**2 + 4.0)) + U[13]*U[17]*(-3.84134961560365*n*(4.0*n**2 + 1.0)/(4.0*n**2 + 4.0) + 15.3653984624146*n/(4.0*n**2 + 4.0)) + 0.05*U[18] - 0.0159154943091895*U[21]*(0.0666666666666667*n**2 + 1.06666666666667)/(4.0*n**2 + 4.0) + U[3]*U[7]*(-3.84134961560365*n*(4.0*n**2 + 1.0)/(4.0*n**2 + 4.0) + 15.3653984624146*n/(4.0*n**2 + 4.0)) - 0.05*U[8] + 0.499701548169216*U[9]*n/(4.0*n**2 + 4.0)
	U_out[9] = U[0]*U[8]*(7.68269923120731*n*(n**2 + 1.0)/(4.0*n**2 + 4.0) - 1.92067480780183*n/(4.0*n**2 + 4.0)) + U[10]*U[18]*(7.68269923120731*n*(n**2 + 1.0)/(4.0*n**2 + 4.0) - 1.92067480780183*n/(4.0*n**2 + 4.0)) + U[13]*U[16]*(3.84134961560365*n*(4.0*n**2 + 1.0)/(4.0*n**2 + 4.0) - 15.3653984624146*n/(4.0*n**2 + 4.0)) + 0.05*U[19] + U[3]*U[6]*(3.84134961560365*n*(4.0*n**2 + 1.0)/(4.0*n**2 + 4.0) - 15.3653984624146*n/(4.0*n**2 + 4.0)) - 0.499701548169216*U[8]*n/(4.0*n**2 + 4.0) - 0.05*U[9]
	U_out[10] = 0.00454545454545455*U[0] + U[10]*(-0.00786487737843552*epsilon - 0.0239816772374912) - 1.09129250443286*U[11]*U[2]*n + 1.09129250443286*U[12]*U[1]*n - 0.873034003546285*U[14]*U[5]*n + 0.873034003546285*U[15]*U[4]*n - 2.18258500886571*U[16]*U[7]*n + 2.18258500886571*U[17]*U[6]*n - 1.74606800709257*U[18]*U[9]*n + 1.74606800709257*U[19]*U[8]*n - 0.000651317236648587*U[21]*(1.33333333333333*n**2 + 21.3333333333333) - 0.000651317236648587*U[23]*(0.533333333333333*n**2 + 34.1333333333333) + U[29]*(0.00176721436265574*epsilon + 0.00673196110231097) + U[31]*(0.000706885745062296*epsilon + 0.00269278444092439) + 0.000468670500616653
	U_out[11] = U[0]*U[12]*(-0.240084350975228*n*(n**2 + 1.0)/(0.2*n**2 + 2.2) - 2.16075915877705*n/(0.2*n**2 + 2.2)) + U[10]*U[2]*(-0.240084350975228*n*(n**2 + 1.0)/(0.2*n**2 + 2.2) + 2.64092786072751*n/(0.2*n**2 + 2.2)) + U[11]*(-0.0173027302325581*epsilon/(0.2*n**2 + 2.2) + 0.014*(-n**2 - 1.0)/(0.2*n**2 + 2.2) - 0.0387596899224806/(0.2*n**2 + 2.2)) + 0.0499701548169216*U[12]*n/(0.2*n**2 + 2.2) + U[13]*U[5]*(-0.384134961560365*n*(n**2 + 4.0)/(0.2*n**2 + 2.2) + 5.37788946184511*n/(0.2*n**2 + 2.2)) + U[14]*U[7]*(0.3*n*(n**2 + 4.0)/(0.2*n**2 + 2.2) + 3.0*n/(0.2*n**2 + 2.2) + 0.2*(-6.0*n**3 - 1.5*n)/(0.2*n**2 + 2.2)) + U[15]*U[3]*(-0.384134961560365*n*(n**2 + 4.0)/(0.2*n**2 + 2.2) - 2.30480976936219*n/(0.2*n**2 + 2.2)) + U[15]*U[6]*(-0.3*n*(n**2 + 4.0)/(0.2*n**2 + 2.2) - 3.0*n/(0.2*n**2 + 2.2) + 0.2*(6.0*n**3 + 1.5*n)/(0.2*n**2 + 2.2)) + U[16]*U[5]*(-0.3*n*(n**2 + 4.0)/(0.2*n**2 + 2.2) + 3.0*n/(0.2*n**2 + 2.2) + 0.2*(6.0*n**3 + 1.5*n)/(0.2*n**2 + 2.2)) + U[17]*U[4]*(0.3*n*(n**2 + 4.0)/(0.2*n**2 + 2.2) - 3.0*n/(0.2*n**2 + 2.2) + 0.2*(-6.0*n**3 - 1.5*n)/(0.2*n**2 + 2.2)) - 0.01*U[1]*(-n**2 - 1.0)/(0.2*n**2 + 2.2) + 0.00318309886183791*U[20]*(0.333333333333333*n**2 + 1.33333333333333)/(0.2*n**2 + 2.2) + U[28]*(-0.00215916979847569*epsilon/(0.2*n**2 + 2.2) - 0.00822506165849588/(0.2*n**2 + 2.2))
	U_out[12] = U[0]*U[11]*(0.240084350975228*n*(n**2 + 1.0)/(0.2*n**2 + 2.2) + 2.16075915877705*n/(0.2*n**2 + 2.2)) + U[10]*U[1]*(0.240084350975228*n*(n**2 + 1.0)/(0.2*n**2 + 2.2) - 2.64092786072751*n/(0.2*n**2 + 2.2)) - 0.0499701548169216*U[11]*n/(0.2*n**2 + 2.2) + U[12]*(-0.0173027302325581*epsilon/(0.2*n**2 + 2.2) + 0.014*(-n**2 - 1.0)/(0.2*n**2 + 2.2) - 0.0387596899224806/(0.2*n**2 + 2.2)) + U[13]*U[4]*(0.384134961560365*n*(n**2 + 4.0)/(0.2*n**2 + 2.2) - 5.37788946184511*n/(0.2*n**2 + 2.2)) + U[14]*U[3]*(0.384134961560365*n*(n**2 + 4.0)/(0.2*n**2 + 2.2) + 2.30480976936219*n/(0.2*n**2 + 2.2)) + U[14]*U[6]*(-0.3*n*(n**2 + 4.0)/(0.2*n**2 + 2.2) - 3.0*n/(0.2*n**2 + 2.2) + 0.2*(6.0*n**3 + 1.5*n)/(0.2*n**2 + 2.2)) + U[15]*U[7]*(-0.3*n*(n**2 + 4.0)/(0.2*n**2 + 2.2) - 3.0*n/(0.2*n**2 + 2.2) + 0.2*(6.0*n**3 + 1.5*n)/(0.2*n**2 + 2.2)) + U[16]*U[4]*(-0.3*n*(n**2 + 4.0)/(0.2*n**2 + 2.2) + 3.0*n/(0.2*n**2 + 2.2) + 0.2*(6.0*n**3 + 1.5*n)/(0.2*n**2 + 2.2)) + U[17]*U[5]*(-0.3*n*(n**2 + 4.0)/(0.2*n**2 + 2.2) + 3.0*n/(0.2*n**2 + 2.2) + 0.2*(6.0*n**3 + 1.5*n)/(0.2*n**2 + 2.2)) + 0.01*U[24]*(-1.0*n**2 - 1.0)/(0.2*n**2 + 2.2) - 0.01*U[2]*(-n**2 - 1.0)/(0.2*n**2 + 2.2) + U[32]*(0.00508742398255814*epsilon/(0.2*n**2 + 2.2) + 0.0193798449612403/(0.2*n**2 + 2.2))
	U_out[13] = U[11]*U[5]*(-0.13719105770013*n*(n**2 + 1.0) + 0.13719105770013*n*(n**2 + 4.0) - 1.3719105770013*n) + U[12]*U[4]*(0.13719105770013*n*(n**2 + 1.0) - 0.13719105770013*n*(n**2 + 4.0) + 1.3719105770013*n) + U[13]*(-0.00617954651162791*epsilon - 0.0338427464008859) + U[14]*U[2]*(0.13719105770013*n*(n**2 + 1.0) - 0.13719105770013*n*(n**2 + 4.0) - 1.3719105770013*n) + U[15]*U[1]*(-0.13719105770013*n*(n**2 + 1.0) + 0.13719105770013*n*(n**2 + 4.0) + 1.3719105770013*n) + U[16]*U[9]*(1.09752846160104*n*(n**2 + 1.0) - 0.274382115400261*n*(4.0*n**2 + 1.0) - 2.74382115400261*n) + U[17]*U[8]*(-1.09752846160104*n*(n**2 + 1.0) + 0.274382115400261*n*(4.0*n**2 + 1.0) + 2.74382115400261*n) + U[18]*U[7]*(-1.09752846160104*n*(n**2 + 1.0) + 0.274382115400261*n*(4.0*n**2 + 1.0) - 2.74382115400261*n) + U[19]*U[6]*(1.09752846160104*n*(n**2 + 1.0) - 0.274382115400261*n*(4.0*n**2 + 1.0) + 2.74382115400261*n) + 0.000511749257366747*U[20]*(0.666666666666667*n**2 + 2.66666666666667) - 0.000511749257366747*U[22]*(1.2*n**2 + 43.2) + U[28]*(-0.000694262785329041*epsilon - 0.00264469900447931) + U[30]*(0.00124967301359227*epsilon + 0.00476045820806276) + 0.0142857142857143*U[3]
	U_out[14] = U[0]*U[15]*(-0.192067480780183*n*(n**2 + 4.0)/(0.2*n**2 + 2.8) - 1.72860732702164*n/(0.2*n**2 + 2.8)) + U[10]*U[5]*(-0.192067480780183*n*(n**2 + 4.0)/(0.2*n**2 + 2.8) + 2.11274228858201*n/(0.2*n**2 + 2.8)) + U[11]*U[7]*(-0.3*n*(n**2 + 1.0)/(0.2*n**2 + 2.8) - 3.0*n/(0.2*n**2 + 2.8) + 0.2*(6.0*n**3 + 1.5*n)/(0.2*n**2 + 2.8)) + U[12]*U[3]*(-0.384134961560365*n*(n**2 + 1.0)/(0.2*n**2 + 2.8) - 2.30480976936219*n/(0.2*n**2 + 2.8)) + U[12]*U[6]*(0.3*n*(n**2 + 1.0)/(0.2*n**2 + 2.8) + 3.0*n/(0.2*n**2 + 2.8) + 0.2*(-6.0*n**3 - 1.5*n)/(0.2*n**2 + 2.8)) + U[13]*U[2]*(-0.384134961560365*n*(n**2 + 1.0)/(0.2*n**2 + 2.8) + 5.37788946184511*n/(0.2*n**2 + 2.8)) + U[14]*(-0.0173027302325581*epsilon/(0.2*n**2 + 2.8) + 0.014*(-n**2 - 4.0)/(0.2*n**2 + 2.8) - 0.0387596899224806/(0.2*n**2 + 2.8)) + 0.0499701548169216*U[15]*n/(0.2*n**2 + 2.8) + U[16]*U[2]*(0.3*n*(n**2 + 1.0)/(0.2*n**2 + 2.8) - 3.0*n/(0.2*n**2 + 2.8) + 0.2*(-6.0*n**3 - 1.5*n)/(0.2*n**2 + 2.8)) + U[17]*U[1]*(-0.3*n*(n**2 + 1.0)/(0.2*n**2 + 2.8) + 3.0*n/(0.2*n**2 + 2.8) + 0.2*(6.0*n**3 + 1.5*n)/(0.2*n**2 + 2.8)) + 0.00318309886183791*U[21]*(0.333333333333333*n**2 + 5.33333333333333)/(0.2*n**2 + 2.8) + U[29]*(-0.00215916979847569*epsilon/(0.2*n**2 + 2.8) - 0.00822506165849588/(0.2*n**2 + 2.8)) - 0.01*U[4]*(-n**2 - 4.0)/(0.2*n**2 + 2.8)
	U_out[15] = U[0]*U[14]*(0.192067480780183*n*(n**2 + 4.0)/(0.2*n**2 + 2.8) + 1.72860732702164*n/(0.2*n**2 + 2.8)) + U[10]*U[4]*(0.192067480780183*n*(n**2 + 4.0)/(0.2*n**2 + 2.8) - 2.11274228858201*n/(0.2*n**2 + 2.8)) + U[11]*U[3]*(0.384134961560365*n*(n**2 + 1.0)/(0.2*n**2 + 2.8) + 2.30480976936219*n/(0.2*n**2 + 2.8)) + U[11]*U[6]*(0.3*n*(n**2 + 1.0)/(0.2*n**2 + 2.8) + 3.0*n/(0.2*n**2 + 2.8) + 0.2*(-6.0*n**3 - 1.5*n)/(0.2*n**2 + 2.8)) + U[12]*U[7]*(0.3*n*(n**2 + 1.0)/(0.2*n**2 + 2.8) + 3.0*n/(0.2*n**2 + 2.8) + 0.2*(-6.0*n**3 - 1.5*n)/(0.2*n**2 + 2.8)) + U[13]*U[1]*(0.384134961560365*n*(n**2 + 1.0)/(0.2*n**2 + 2.8) - 5.37788946184511*n/(0.2*n**2 + 2.8)) - 0.0499701548169216*U[14]*n/(0.2*n**2 + 2.8) + U[15]*(-0.0173027302325581*epsilon/(0.2*n**2 + 2.8) + 0.014*(-n**2 - 4.0)/(0.2*n**2 + 2.8) - 0.0387596899224806/(0.2*n**2 + 2.8)) + U[16]*U[1]*(0.3*n*(n**2 + 1.0)/(0.2*n**2 + 2.8) - 3.0*n/(0.2*n**2 + 2.8) + 0.2*(-6.0*n**3 - 1.5*n)/(0.2*n**2 + 2.8)) + U[17]*U[2]*(0.3*n*(n**2 + 1.0)/(0.2*n**2 + 2.8) - 3.0*n/(0.2*n**2 + 2.8) + 0.2*(-6.0*n**3 - 1.5*n)/(0.2*n**2 + 2.8)) + 0.01*U[25]*(-1.0*n**2 - 4.0)/(0.2*n**2 + 2.8) + U[33]*(0.00508742398255814*epsilon/(0.2*n**2 + 2.8) + 0.0193798449612403/(0.2*n**2 + 2.8)) - 0.01*U[5]*(-n**2 - 4.0)/(0.2*n**2 + 2.8)
	U_out[16] = U[0]*U[17]*(-0.480168701950457*n*(4.0*n**2 + 1.0)/(0.8*n**2 + 2.2) - 4.32151831755411*n/(0.8*n**2 + 2.2)) + U[10]*U[7]*(-0.480168701950457*n*(4.0*n**2 + 1.0)/(0.8*n**2 + 2.2) + 5.28185572145502*n/(0.8*n**2 + 2.2)) + U[11]*U[5]*(-0.3*n*(n**2 + 1.0)/(0.8*n**2 + 2.2) + 0.3*n*(n**2 + 4.0)/(0.8*n**2 + 2.2) - 3.0*n/(0.8*n**2 + 2.2)) + U[12]*U[4]*(-0.3*n*(n**2 + 1.0)/(0.8*n**2 + 2.2) + 0.3*n*(n**2 + 4.0)/(0.8*n**2 + 2.2) - 3.0*n/(0.8*n**2 + 2.2)) + U[13]*U[9]*(-3.07307969248292*n*(n**2 + 1.0)/(0.8*n**2 + 2.2) + 10.7557789236902*n/(0.8*n**2 + 2.2)) + U[14]*U[2]*(-0.3*n*(n**2 + 1.0)/(0.8*n**2 + 2.2) + 0.3*n*(n**2 + 4.0)/(0.8*n**2 + 2.2) + 3.0*n/(0.8*n**2 + 2.2)) + U[15]*U[1]*(-0.3*n*(n**2 + 1.0)/(0.8*n**2 + 2.2) + 0.3*n*(n**2 + 4.0)/(0.8*n**2 + 2.2) + 3.0*n/(0.8*n**2 + 2.2)) + U[16]*(-0.0173027302325581*epsilon/(0.8*n**2 + 2.2) + 0.014*(-4.0*n**2 - 1.0)/(0.8*n**2 + 2.2) - 0.0387596899224806/(0.8*n**2 + 2.2)) + 0.0999403096338432*U[17]*n/(0.8*n**2 + 2.2) + U[19]*U[3]*(-3.07307969248292*n*(n**2 + 1.0)/(0.8*n**2 + 2.2) - 4.60961953872438*n/(0.8*n**2 + 2.2)) + 0.00318309886183791*U[20]*(0.0666666666666667*n**2 + 0.266666666666667)/(0.8*n**2 + 2.2) + U[28]*(-0.000431833959695138*epsilon/(0.8*n**2 + 2.2) - 0.00164501233169918/(0.8*n**2 + 2.2)) - 0.01*U[6]*(-4.0*n**2 - 1.0)/(0.8*n**2 + 2.2)
	U_out[17] = U[0]*U[16]*(0.480168701950457*n*(4.0*n**2 + 1.0)/(0.8*n**2 + 2.2) + 4.32151831755411*n/(0.8*n**2 + 2.2)) + U[10]*U[6]*(0.480168701950457*n*(4.0*n**2 + 1.0)/(0.8*n**2 + 2.2) - 5.28185572145502*n/(0.8*n**2 + 2.2)) + U[11]*U[4]*(0.3*n*(n**2 + 1.0)/(0.8*n**2 + 2.2) - 0.3*n*(n**2 + 4.0)/(0.8*n**2 + 2.2) + 3.0*n/(0.8*n**2 + 2.2)) + U[12]*U[5]*(-0.3*n*(n**2 + 1.0)/(0.8*n**2 + 2.2) + 0.3*n*(n**2 + 4.0)/(0.8*n**2 + 2.2) - 3.0*n/(0.8*n**2 + 2.2)) + U[13]*U[8]*(3.07307969248292*n*(n**2 + 1.0)/(0.8*n**2 + 2.2) - 10.7557789236902*n/(0.8*n**2 + 2.2)) + U[14]*U[1]*(0.3*n*(n**2 + 1.0)/(0.8*n**2 + 2.2) - 0.3*n*(n**2 + 4.0)/(0.8*n**2 + 2.2) - 3.0*n/(0.8*n**2 + 2.2)) + U[15]*U[2]*(-0.3*n*(n**2 + 1.0)/(0.8*n**2 + 2.2) + 0.3*n*(n**2 + 4.0)/(0.8*n**2 + 2.2) + 3.0*n/(0.8*n**2 + 2.2)) - 0.0999403096338432*U[16]*n/(0.8*n**2 + 2.2) + U[17]*(-0.0173027302325581*epsilon/(0.8*n**2 + 2.2) + 0.014*(-4.0*n**2 - 1.0)/(0.8*n**2 + 2.2) - 0.0387596899224806/(0.8*n**2 + 2.2)) + U[18]*U[3]*(3.07307969248292*n*(n**2 + 1.0)/(0.8*n**2 + 2.2) + 4.60961953872438*n/(0.8*n**2 + 2.2)) - 0.01*U[7]*(-4.0*n**2 - 1.0)/(0.8*n**2 + 2.2)
	U_out[18] = U[0]*U[19]*(-0.768269923120731*n*(n**2 + 1.0)/(0.4*n**2 + 1.4) - 1.72860732702164*n/(0.4*n**2 + 1.4)) + U[10]*U[9]*(-0.768269923120731*n*(n**2 + 1.0)/(0.4*n**2 + 1.4) + 2.11274228858201*n/(0.4*n**2 + 1.4)) + U[13]*U[7]*(-0.384134961560365*n*(4.0*n**2 + 1.0)/(0.4*n**2 + 1.4) + 5.37788946184511*n/(0.4*n**2 + 1.4)) + U[17]*U[3]*(-0.384134961560365*n*(4.0*n**2 + 1.0)/(0.4*n**2 + 1.4) - 2.30480976936219*n/(0.4*n**2 + 1.4)) + U[18]*(-0.00865136511627907*epsilon/(0.4*n**2 + 1.4) + 0.007*(-4.0*n**2 - 4.0)/(0.4*n**2 + 1.4) - 0.0193798449612403/(0.4*n**2 + 1.4)) + 0.0499701548169216*U[19]*n/(0.4*n**2 + 1.4) + 0.00159154943091895*U[21]*(0.0666666666666667*n**2 + 1.06666666666667)/(0.4*n**2 + 1.4) + U[29]*(-0.000215916979847569*epsilon/(0.4*n**2 + 1.4) - 0.000822506165849588/(0.4*n**2 + 1.4)) - 0.005*U[8]*(-4.0*n**2 - 4.0)/(0.4*n**2 + 1.4)
	U_out[19] = U[0]*U[18]*(0.768269923120731*n*(n**2 + 1.0)/(0.4*n**2 + 1.4) + 1.72860732702164*n/(0.4*n**2 + 1.4)) + U[10]*U[8]*(0.768269923120731*n*(n**2 + 1.0)/(0.4*n**2 + 1.4) - 2.11274228858201*n/(0.4*n**2 + 1.4)) + U[13]*U[6]*(0.384134961560365*n*(4.0*n**2 + 1.0)/(0.4*n**2 + 1.4) - 5.37788946184511*n/(0.4*n**2 + 1.4)) + U[16]*U[3]*(0.384134961560365*n*(4.0*n**2 + 1.0)/(0.4*n**2 + 1.4) + 2.30480976936219*n/(0.4*n**2 + 1.4)) - 0.0499701548169216*U[18]*n/(0.4*n**2 + 1.4) + U[19]*(-0.00865136511627907*epsilon/(0.4*n**2 + 1.4) + 0.007*(-4.0*n**2 - 4.0)/(0.4*n**2 + 1.4) - 0.0193798449612403/(0.4*n**2 + 1.4)) - 0.005*U[9]*(-4.0*n**2 - 4.0)/(0.4*n**2 + 1.4)
	U_out[20] = 0.000637442278533431*U[11]*(n**2 + 1.0)/(3.875*n**2 + 26992.8730718743) + 0.00229559873588804*U[13]/(3.875*n**2 + 26992.8730718743) + 0.000478081708900073*U[16]*(1.06666666666667*n**2 + 0.266666666666667)/(3.875*n**2 + 26992.8730718743) - 0.000637442278533431*U[1]*(n**2 + 1.0)/(3.875*n**2 + 26992.8730718743) + 0.00300387596899225*U[20]*(-0.25*n**2 - 1.0)/(3.875*n**2 + 26992.8730718743) + U[21]*U[24]*(-15.5*n*(0.1875*n**2 + 3.0)/(3.875*n**2 + 26992.8730718743) + 11.625*n*(n**2 + 1.0)/(3.875*n**2 + 26992.8730718743)) + U[21]*U[26]*(15.5*n*(0.0625*n**2 + 1.0)/(3.875*n**2 + 26992.8730718743) - 15.5*n*(0.25*n**2 + 2.25)/(3.875*n**2 + 26992.8730718743)) + U[22]*U[25]*(-15.5*n*(0.25*n**2 + 9.0)/(3.875*n**2 + 26992.8730718743) + 15.5*n*(1.0*n**2 + 4.0)/(3.875*n**2 + 26992.8730718743)) + U[22]*U[27]*(15.5*n*(0.125*n**2 + 4.5)/(3.875*n**2 + 26992.8730718743) - 15.5*n*(0.5*n**2 + 8.0)/(3.875*n**2 + 26992.8730718743)) + U[23]*U[26]*(-15.5*n*(0.3125*n**2 + 20.0)/(3.875*n**2 + 26992.8730718743) + 15.5*n*(1.25*n**2 + 11.25)/(3.875*n**2 + 26992.8730718743)) - 1.64361941021061*U[24]*n/(3.875*n**2 + 26992.8730718743) - 0.00229559873588804*U[3]/(3.875*n**2 + 26992.8730718743) - 0.000478081708900073*U[6]*(1.06666666666667*n**2 + 0.266666666666667)/(3.875*n**2 + 26992.8730718743)
	U_out[21] = 0.00114779936794402*U[0]/(3.875*n**2 + 27039.3730718743) - 0.00114779936794402*U[10]/(3.875*n**2 + 27039.3730718743) + 0.000478081708900073*U[14]*(1.33333333333333*n**2 + 5.33333333333333)/(3.875*n**2 + 27039.3730718743) + 0.000509953822826745*U[18]*(n**2 + 1.0)/(3.875*n**2 + 27039.3730718743) + U[20]*U[24]*(15.5*n*(0.1875*n**2 + 0.75)/(3.875*n**2 + 27039.3730718743) - 11.625*n*(n**2 + 1.0)/(3.875*n**2 + 27039.3730718743)) + U[20]*U[26]*(-15.5*n*(0.0625*n**2 + 0.25)/(3.875*n**2 + 27039.3730718743) + 15.5*n*(0.25*n**2 + 2.25)/(3.875*n**2 + 27039.3730718743)) + 0.00300387596899225*U[21]*(-0.25*n**2 - 4.0)/(3.875*n**2 + 27039.3730718743) + U[22]*U[24]*(-15.5*n*(0.3125*n**2 + 11.25)/(3.875*n**2 + 27039.3730718743) + 19.375*n*(n**2 + 1.0)/(3.875*n**2 + 27039.3730718743)) + U[23]*U[25]*(-15.5*n*(0.375*n**2 + 24.0)/(3.875*n**2 + 27039.3730718743) + 15.5*n*(1.5*n**2 + 6.0)/(3.875*n**2 + 27039.3730718743)) - 1.64361941021061*U[25]*n/(3.875*n**2 + 27039.3730718743) - 0.000478081708900073*U[4]*(1.33333333333333*n**2 + 5.33333333333333)/(3.875*n**2 + 27039.3730718743) - 0.000509953822826745*U[8]*(n**2 + 1.0)/(3.875*n**2 + 27039.3730718743)
	U_out[22] = -0.00413207772459848*U[13]/(3.875*n**2 + 27116.8730718743) + U[20]*U[25]*(15.5*n*(0.25*n**2 + 1.0)/(3.875*n**2 + 27116.8730718743) - 15.5*n*(1.0*n**2 + 4.0)/(3.875*n**2 + 27116.8730718743)) + U[20]*U[27]*(-15.5*n*(0.125*n**2 + 0.5)/(3.875*n**2 + 27116.8730718743) + 15.5*n*(0.5*n**2 + 8.0)/(3.875*n**2 + 27116.8730718743)) + U[21]*U[24]*(15.5*n*(0.3125*n**2 + 5.0)/(3.875*n**2 + 27116.8730718743) - 19.375*n*(n**2 + 1.0)/(3.875*n**2 + 27116.8730718743)) + 0.00300387596899225*U[22]*(-0.25*n**2 - 9.0)/(3.875*n**2 + 27116.8730718743) + U[23]*U[24]*(-15.5*n*(0.4375*n**2 + 28.0)/(3.875*n**2 + 27116.8730718743) + 27.125*n*(n**2 + 1.0)/(3.875*n**2 + 27116.8730718743)) - 1.64361941021061*U[26]*n/(3.875*n**2 + 27116.8730718743) + 0.00413207772459848*U[3]/(3.875*n**2 + 27116.8730718743)
	U_out[23] = 0.000459119747177608*U[0]/(3.875*n**2 + 27225.3730718743) - 0.000459119747177608*U[10]/(3.875*n**2 + 27225.3730718743) + U[20]*U[26]*(15.5*n*(0.3125*n**2 + 1.25)/(3.875*n**2 + 27225.3730718743) - 15.5*n*(1.25*n**2 + 11.25)/(3.875*n**2 + 27225.3730718743)) + U[21]*U[25]*(15.5*n*(0.375*n**2 + 6.0)/(3.875*n**2 + 27225.3730718743) - 15.5*n*(1.5*n**2 + 6.0)/(3.875*n**2 + 27225.3730718743)) + U[22]*U[24]*(15.5*n*(0.4375*n**2 + 15.75)/(3.875*n**2 + 27225.3730718743) - 27.125*n*(n**2 + 1.0)/(3.875*n**2 + 27225.3730718743)) + 0.00300387596899225*U[23]*(-0.25*n**2 - 16.0)/(3.875*n**2 + 27225.3730718743) - 1.64361941021061*U[27]*n/(3.875*n**2 + 27225.3730718743)
	U_out[24] = 0.00150193798449612*U[12]*(-1.0*n**2 - 1.0)/(15.5*n**2 + 26992.8730718743) + U[20]*U[21]*(-15.5*n*(0.1875*n**2 + 0.75)/(15.5*n**2 + 26992.8730718743) + 15.5*n*(0.1875*n**2 + 3.0)/(15.5*n**2 + 26992.8730718743)) + 1.64361941021061*U[20]*n/(15.5*n**2 + 26992.8730718743) + U[21]*U[22]*(-15.5*n*(0.3125*n**2 + 5.0)/(15.5*n**2 + 26992.8730718743) + 15.5*n*(0.3125*n**2 + 11.25)/(15.5*n**2 + 26992.8730718743)) + U[22]*U[23]*(-15.5*n*(0.4375*n**2 + 15.75)/(15.5*n**2 + 26992.8730718743) + 15.5*n*(0.4375*n**2 + 28.0)/(15.5*n**2 + 26992.8730718743)) + 0.00300387596899225*U[24]*(-1.0*n**2 - 1.0)/(15.5*n**2 + 26992.8730718743) - 0.00150193798449612*U[2]*(-1.0*n**2 - 1.0)/(15.5*n**2 + 26992.8730718743)
	U_out[25] = 0.00150193798449612*U[15]*(-1.0*n**2 - 4.0)/(15.5*n**2 + 27039.3730718743) + U[20]*U[22]*(-15.5*n*(0.25*n**2 + 1.0)/(15.5*n**2 + 27039.3730718743) + 15.5*n*(0.25*n**2 + 9.0)/(15.5*n**2 + 27039.3730718743)) + U[21]*U[23]*(-15.5*n*(0.375*n**2 + 6.0)/(15.5*n**2 + 27039.3730718743) + 15.5*n*(0.375*n**2 + 24.0)/(15.5*n**2 + 27039.3730718743)) + 1.64361941021061*U[21]*n/(15.5*n**2 + 27039.3730718743) + 0.00300387596899225*U[25]*(-1.0*n**2 - 4.0)/(15.5*n**2 + 27039.3730718743) - 0.00150193798449612*U[5]*(-1.0*n**2 - 4.0)/(15.5*n**2 + 27039.3730718743)
	U_out[26] = U[20]*U[21]*(15.5*n*(0.0625*n**2 + 0.25)/(15.5*n**2 + 27116.8730718743) - 15.5*n*(0.0625*n**2 + 1.0)/(15.5*n**2 + 27116.8730718743)) + U[20]*U[23]*(-15.5*n*(0.3125*n**2 + 1.25)/(15.5*n**2 + 27116.8730718743) + 15.5*n*(0.3125*n**2 + 20.0)/(15.5*n**2 + 27116.8730718743)) + 1.64361941021061*U[22]*n/(15.5*n**2 + 27116.8730718743) + 0.00300387596899225*U[26]*(-1.0*n**2 - 9.0)/(15.5*n**2 + 27116.8730718743)
	U_out[27] = U[20]*U[22]*(15.5*n*(0.125*n**2 + 0.5)/(15.5*n**2 + 27225.3730718743) - 15.5*n*(0.125*n**2 + 4.5)/(15.5*n**2 + 27225.3730718743)) + 1.64361941021061*U[23]*n/(15.5*n**2 + 27225.3730718743) + 0.00300387596899225*U[27]*(-1.0*n**2 - 16.0)/(15.5*n**2 + 27225.3730718743)
	U_out[28] = U[11]*(-0.000183587669699814*epsilon - 0.000822506165849588) + U[13]*(-0.000165286974476004*epsilon - 0.000740515721254207) + U[16]*(-3.67175339399628e-5*epsilon - 0.000164501233169918) + 0.75*U[21]*U[32]*n - 0.25*U[21]*U[34]*n + 1.0*U[22]*U[33]*n - 0.5*U[22]*U[35]*n + 1.25*U[23]*U[34]*n - 0.75*U[24]*U[29]*n - 1.0*U[25]*U[30]*n + 0.25*U[26]*U[29]*n - 1.25*U[26]*U[31]*n + 0.5*U[27]*U[30]*n - 0.00122336344718992*U[28]
	U_out[29] = U[10]*(0.000330573948952008*epsilon + 0.00148103144250841) + U[14]*(-0.000183587669699814*epsilon - 0.000822506165849588) + U[18]*(-3.67175339399628e-5*epsilon - 0.000164501233169918) - 0.75*U[20]*U[32]*n + 0.25*U[20]*U[34]*n + 1.25*U[22]*U[32]*n + 1.5*U[23]*U[33]*n + 0.75*U[24]*U[28]*n - 1.25*U[24]*U[30]*n - 1.5*U[25]*U[31]*n - 0.25*U[26]*U[28]*n - 0.00122336344718992*U[29] + 0.000137893034416115
	U_out[30] = U[13]*(0.000297516554056807*epsilon + 0.00133292829825757) - 1.0*U[20]*U[33]*n + 0.5*U[20]*U[35]*n - 1.25*U[21]*U[32]*n + 1.75*U[23]*U[32]*n + 1.25*U[24]*U[29]*n - 1.75*U[24]*U[31]*n + 1.0*U[25]*U[28]*n - 0.5*U[27]*U[28]*n - 0.00122336344718992*U[30]
	U_out[31] = U[10]*(0.000132229579580803*epsilon + 0.000592412577003366) - 1.25*U[20]*U[34]*n - 1.5*U[21]*U[33]*n - 1.75*U[22]*U[32]*n + 1.75*U[24]*U[30]*n + 1.5*U[25]*U[29]*n + 1.25*U[26]*U[28]*n - 0.00122336344718992*U[31] + 5.51572137664459e-5
	U_out[32] = U[12]*(0.000432568255813953*epsilon + 0.00193798449612403) + 0.75*U[20]*U[29]*n - 0.75*U[21]*U[28]*n + 1.25*U[21]*U[30]*n - 1.25*U[22]*U[29]*n + 1.75*U[22]*U[31]*n - 1.75*U[23]*U[30]*n - 0.00122336344718992*U[32]
	U_out[33] = U[15]*(0.000432568255813953*epsilon + 0.00193798449612403) + 1.0*U[20]*U[30]*n + 1.5*U[21]*U[31]*n - 1.0*U[22]*U[28]*n - 1.5*U[23]*U[29]*n - 0.00122336344718992*U[33]
	U_out[34] = -0.25*U[20]*U[29]*n + 1.25*U[20]*U[31]*n + 0.25*U[21]*U[28]*n - 1.25*U[23]*U[28]*n - 0.00122336344718992*U[34]
	U_out[35] = -0.5*U[20]*U[30]*n + 0.5*U[22]*U[28]*n - 0.00122336344718992*U[35]
	return U_out

In [ ]:
offset = 1 if model_parameters.dynamic_T else 0

In [ ]:
model_parameters.ndim

## Comparing with numerical results

In [ ]:
from qgs.integrators.integrator import RungeKuttaIntegrator, RungeKuttaTglsIntegrator
import matplotlib.pyplot as plt

In [ ]:
f_num, Df, ten_num = create_tendencies(model_parameters, return_qgtensor=True)

In [ ]:
integrator = RungeKuttaIntegrator()
integrator.set_func(f)

In [ ]:
integrator_num = RungeKuttaIntegrator()
integrator_num.set_func(f_num)

In [ ]:
# ICs calculated from long transient

ic = np.array([ 0.03099396, -0.00496147,  0.00173704, -0.00135697, -0.00772211,
       -0.01966693,  0.0133646 ,  0.001414  ,  0.01117828, -0.00041981,
        0.0296539 , -0.00059846, -0.0054299 ,  0.00265086, -0.00813694,
       -0.01703874,  0.00775618, -0.00437281,  0.00628166, -0.00235651,
       -0.00471975,  0.00143941, -0.00162457, -0.00087876,  0.00399296,
       -0.00265117, -0.00265747,  0.00141623, -0.0210313 ,  0.05039241,
        0.01445278,  0.00645707, -0.04806996, -0.06529819, -0.00455508,
        0.04942686])

In [ ]:
%%time
integrator.integrate(0., 1000000., 0.1, ic=ic, write_steps=5)
reference_time, reference_traj = integrator.get_trajectories()

In [ ]:
%%time
integrator_num.integrate(0., 1000000., 0.1, ic=ic, write_steps=5)
reference_time_num, reference_traj_num = integrator_num.get_trajectories()

In [ ]:
varx = 21
vary = 29
plt.figure(figsize=(12, 10))

plt.plot(reference_traj[varx], reference_traj[vary], marker='o', ms=0.07, ls='')
plt.plot(reference_traj_num[varx], reference_traj_num[vary], marker='o', ms=0.07, ls='')

plt.xlabel('$'+model_parameters.latex_var_string[varx]+'$')
plt.ylabel('$'+model_parameters.latex_var_string[vary]+'$');

In [ ]:
plt.figure(figsize=(12, 8))

plt.plot(reference_traj_num[varx, :])
plt.plot(reference_traj[varx, :])
plt.show()